## Import libraries

In [139]:
import os 
import csv
import numpy as np
import pandas as pd
import random # library to generate random numbers
np.random.seed(seed=42)
import math

#### Import data

In [140]:
data1 = pd.read_csv("cities_and_distances.csv",header=0,index_col=0)
data1

,Bangalore,Jaipur,Varanasi,Udaipur,Delhi,Chennai,Mysore,Agra,Kolkata,Mumbai,Hyderabad
Bangalore,0,2026,1825,1724,2166,346,149,1923,1862,980,569
Jaipur,2026,0,849,393,287,2108,2121,240,1520,1146,1483
Varanasi,1825,849,0,1154,834,1862,1976,614,683,1524,1237
Udaipur,1724,393,1154,0,661,2077,1809,634,1825,753,1349
Delhi,2166,287,834,661,0,2203,2317,233,1498,1415,1578
Chennai,346,2108,1862,2077,2203,0,482,1957,1669,1345,627
Mysore,149,2121,1976,1809,2317,482,0,2072,2012,1064,718
Agra,1923,240,614,634,233,1957,2072,0,1280,1323,1334
Kolkata,1862,1520,683,1825,1498,1669,2012,1280,0,2053,1493
Mumbai,980,1146,1524,753,1415,1345,1064,1323,2053,0,711


#### Starting Point is Bangalore

In [141]:
starting_point = "Bangalore"

In [142]:
data1.columns

Index(['Bangalore', 'Jaipur', 'Varanasi', 'Udaipur', 'Delhi', 'Chennai',
       'Mysore', 'Agra', 'Kolkata', 'Mumbai', 'Hyderabad'],
      dtype='object')

In [143]:
data1.shape

(11, 11)

###### The number of possible solutions are: 10! = 3628800
We can generate all possible solutions and try to find the best of those , but this is a time taking process and may not be feasible for big problems.
Genetic algorithm approach takes a randominzed search approach to find the best solution, with enough number of iterations, mutation and cross over, GA can reach the global minima for most of the problems.

##### Map places to integers

In [144]:
cities_mapping = {}

for i in enumerate(data1.columns):
    cities_mapping[i[0]] = i[1]
    print(i[0])
    print(i[1])
    print(cities_mapping[i[0]])
        
print("\n")
print(cities_mapping)
print(type(cities_mapping))

0
Bangalore
Bangalore
1
Jaipur
Jaipur
2
Varanasi
Varanasi
3
Udaipur
Udaipur
4
Delhi
Delhi
5
Chennai
Chennai
6
Mysore
Mysore
7
Agra
Agra
8
Kolkata
Kolkata
9
Mumbai
Mumbai
10
Hyderabad
Hyderabad


{0: 'Bangalore', 1: 'Jaipur', 2: 'Varanasi', 3: 'Udaipur', 4: 'Delhi', 5: 'Chennai', 6: 'Mysore', 7: 'Agra', 8: 'Kolkata', 9: 'Mumbai', 10: 'Hyderabad'}
<class 'dict'>


In [145]:
enumerate(data1.columns)

In [146]:
one_route = [5,6,10,9,1,3,4,7,2,8] # one of the routes

In [147]:
one_route

[5, 6, 10, 9, 1, 3, 4, 7, 2, 8]

## createRandomRoute:: this function is to generate random route

In [148]:
def createRandomRoute():
    """ This function generates a route by sampling numbers """
    my_randoms = random.sample(range(1,11), 10)    # excluding 0 from the range as 0 is the starting point for us i.e. Bangalore
    return my_randoms


In [149]:
temp_route = createRandomRoute()

In [150]:
temp_route

[10, 2, 3, 1, 8, 4, 5, 9, 7, 6]

In [151]:
len(temp_route)

10

In [152]:
data1

,Bangalore,Jaipur,Varanasi,Udaipur,Delhi,Chennai,Mysore,Agra,Kolkata,Mumbai,Hyderabad
Bangalore,0,2026,1825,1724,2166,346,149,1923,1862,980,569
Jaipur,2026,0,849,393,287,2108,2121,240,1520,1146,1483
Varanasi,1825,849,0,1154,834,1862,1976,614,683,1524,1237
Udaipur,1724,393,1154,0,661,2077,1809,634,1825,753,1349
Delhi,2166,287,834,661,0,2203,2317,233,1498,1415,1578
Chennai,346,2108,1862,2077,2203,0,482,1957,1669,1345,627
Mysore,149,2121,1976,1809,2317,482,0,2072,2012,1064,718
Agra,1923,240,614,634,233,1957,2072,0,1280,1323,1334
Kolkata,1862,1520,683,1825,1498,1669,2012,1280,0,2053,1493
Mumbai,980,1146,1524,753,1415,1345,1064,1323,2053,0,711


In [153]:
data1.iloc[0, temp_route[0]]

569

## fitnessFunction :: Function to compute the route cost using the already defined path 

In [154]:
def fitnessFunction(pvRouteMap):
    """This functions takes the route generated and returns the cost incurred."""
    traverseData    = data1.copy() # creating a copy of the original dataframe
    sourcePoint     = 0 # defining a starting point
    stopsCovered    = 0 # setting the number of stops covered to 0
    routeCost       = 0 # setting the initial cost to 0
    pvRouteMap1 = pvRouteMap #+ [pvRouteMap[0]]
    
    # initiate a while loop and calculate the cost for the whole path traversed and return the cost
    while(stopsCovered < len(pvRouteMap1)) :  
        routeCost = routeCost + traverseData.iloc[sourcePoint, pvRouteMap1[stopsCovered]]          
        
        print("routeCost+", sourcePoint, "->",pvRouteMap1[stopsCovered], routeCost)
        print("\n")
        
        """route cost is the sum of the cost incurred from travelling from one point to the next according
        to the route that was generated previously."""
        
        sourcePoint = pvRouteMap1[stopsCovered] # update the source point to the next point in the route
        stopsCovered = stopsCovered+1 #Adding 1 to the stops covered
#         print(sourcePoint)
    routeCost = routeCost + traverseData.iloc[pvRouteMap1[-1],0]

    return(routeCost)

In [155]:
fitnessFunction(temp_route)

routeCost+ 0 -> 10 569


routeCost+ 10 -> 2 1806


routeCost+ 2 -> 3 2960


routeCost+ 3 -> 1 3353


routeCost+ 1 -> 8 4873


routeCost+ 8 -> 4 6371


routeCost+ 4 -> 5 8574


routeCost+ 5 -> 9 9919


routeCost+ 9 -> 7 11242


routeCost+ 7 -> 6 13314




13463

#### We can look at the positions and think of one good route, we will see if our algorithm performs better than this route

In [156]:
print(one_route)

[5, 6, 10, 9, 1, 3, 4, 7, 2, 8]


In [157]:
one_route_cities = []

for i in one_route:
    one_route_cities.append(cities_mapping[i])
    
one_route_cities

['Chennai',
 'Mysore',
 'Hyderabad',
 'Mumbai',
 'Jaipur',
 'Udaipur',
 'Delhi',
 'Agra',
 'Varanasi',
 'Kolkata']

In [158]:
fitnessFunction(one_route)

routeCost+ 0 -> 5 346


routeCost+ 5 -> 6 828


routeCost+ 6 -> 10 1546


routeCost+ 10 -> 9 2257


routeCost+ 9 -> 1 3403


routeCost+ 1 -> 3 3796


routeCost+ 3 -> 4 4457


routeCost+ 4 -> 7 4690


routeCost+ 7 -> 2 5304


routeCost+ 2 -> 8 5987




7849

##  mutateRoute:: this function will generate mutated path where we define mutation policy

In [159]:
def mutateRoute(initPath, mutateFactor):
    """ This functions generates a mutated path, takes an input path and returns a mutated path.
    Mutate factor is the point at which the string(route here) is split and the two parts are swapped."""

    temp1 = initPath[0:mutateFactor]
    print(temp1)
    temp2 = initPath[mutateFactor:len(initPath)]
    print(temp2)
    newPath = temp2 + temp1
    print(newPath)
    return(newPath)

#### Generate a sample path and mutate that path 

In [160]:
temp_route = createRandomRoute()

In [161]:
temp_route

[5, 4, 8, 7, 1, 9, 10, 2, 6, 3]

In [162]:
mutateRoute(temp_route,2)

[5, 4]
[8, 7, 1, 9, 10, 2, 6, 3]
[8, 7, 1, 9, 10, 2, 6, 3, 5, 4]


[8, 7, 1, 9, 10, 2, 6, 3, 5, 4]

In [163]:
fitnessFunction(temp_route)

routeCost+ 0 -> 5 346


routeCost+ 5 -> 4 2549


routeCost+ 4 -> 8 4047


routeCost+ 8 -> 7 5327


routeCost+ 7 -> 1 5567


routeCost+ 1 -> 9 6713


routeCost+ 9 -> 10 7424


routeCost+ 10 -> 2 8661


routeCost+ 2 -> 6 10637


routeCost+ 6 -> 3 12446




14170

In [164]:
print("Total distance travelled to cover the route of {} is {}".format(" => ".join( one_route_cities ),fitnessFunction(one_route)))


routeCost+ 0 -> 5 346


routeCost+ 5 -> 6 828


routeCost+ 6 -> 10 1546


routeCost+ 10 -> 9 2257


routeCost+ 9 -> 1 3403


routeCost+ 1 -> 3 3796


routeCost+ 3 -> 4 4457


routeCost+ 4 -> 7 4690


routeCost+ 7 -> 2 5304


routeCost+ 2 -> 8 5987


Total distance travelled to cover the route of Chennai => Mysore => Hyderabad => Mumbai => Jaipur => Udaipur => Delhi => Agra => Varanasi => Kolkata is 7849


# Function to create Initial (random) Population 

In [165]:
def initialPopCost(ninitpop = 10):
    """This function generates new routes and calculates the cost of the route.
    returns a dictionary of routes and the cost."""
    np.random.seed(seed=42)
    intial_cost = 0
    routeCost = []
    routes = [createRandomRoute() for i in range(0, ninitpop)]
    routes_Cost = [fitnessFunction(i) for i in routes]
    routes_DF = pd.DataFrame([routes,routes_Cost]).transpose()
    routes_DF.columns = ['Route','Cost']
    return(routes_DF)


In [166]:
initial_pop_cost = initialPopCost(10)
initial_pop_cost

routeCost+ 0 -> 1 2026


routeCost+ 1 -> 8 3546


routeCost+ 8 -> 2 4229


routeCost+ 2 -> 7 4843


routeCost+ 7 -> 10 6177


routeCost+ 10 -> 4 7755


routeCost+ 4 -> 3 8416


routeCost+ 3 -> 9 9169


routeCost+ 9 -> 5 10514


routeCost+ 5 -> 6 10996


routeCost+ 0 -> 3 1724


routeCost+ 3 -> 9 2477


routeCost+ 9 -> 4 3892


routeCost+ 4 -> 5 6095


routeCost+ 5 -> 10 6722


routeCost+ 10 -> 6 7440


routeCost+ 6 -> 7 9512


routeCost+ 7 -> 8 10792


routeCost+ 8 -> 1 12312


routeCost+ 1 -> 2 13161


routeCost+ 0 -> 5 346


routeCost+ 5 -> 6 828


routeCost+ 6 -> 1 2949


routeCost+ 1 -> 10 4432


routeCost+ 10 -> 2 5669


routeCost+ 2 -> 9 7193


routeCost+ 9 -> 7 8516


routeCost+ 7 -> 3 9150


routeCost+ 3 -> 4 9811


routeCost+ 4 -> 8 11309


routeCost+ 0 -> 4 2166


routeCost+ 4 -> 3 2827


routeCost+ 3 -> 1 3220


routeCost+ 1 -> 9 4366


routeCost+ 9 -> 7 5689


routeCost+ 7 -> 8 6969


routeCost+ 8 -> 10 8462


routeCost+ 10 -> 2 9699


routeCost+ 2 -> 5 11561


routeCost+ 5

,Route,Cost
0,"[1, 8, 2, 7, 10, 4, 3, 9, 5, 6]",11145
1,"[3, 9, 4, 5, 10, 6, 7, 8, 1, 2]",14986
2,"[5, 6, 1, 10, 2, 9, 7, 3, 4, 8]",13171
3,"[4, 3, 1, 9, 7, 8, 10, 2, 5, 6]",12192
4,"[7, 6, 2, 1, 3, 5, 8, 10, 4, 9]",16425
5,"[1, 10, 3, 4, 9, 2, 6, 5, 8, 7]",15788
6,"[6, 1, 5, 2, 10, 4, 3, 7, 8, 9]",14663
7,"[2, 10, 4, 3, 7, 5, 9, 1, 6, 8]",16378
8,"[10, 1, 3, 8, 4, 2, 5, 7, 9, 6]",12957
9,"[6, 3, 10, 7, 4, 1, 5, 9, 8, 2]",13175


In [167]:
sorted_init_pop = initial_pop_cost.sort_values(['Cost'])
sorted_init_pop

,Route,Cost
0,"[1, 8, 2, 7, 10, 4, 3, 9, 5, 6]",11145
3,"[4, 3, 1, 9, 7, 8, 10, 2, 5, 6]",12192
8,"[10, 1, 3, 8, 4, 2, 5, 7, 9, 6]",12957
2,"[5, 6, 1, 10, 2, 9, 7, 3, 4, 8]",13171
9,"[6, 3, 10, 7, 4, 1, 5, 9, 8, 2]",13175
6,"[6, 1, 5, 2, 10, 4, 3, 7, 8, 9]",14663
1,"[3, 9, 4, 5, 10, 6, 7, 8, 1, 2]",14986
5,"[1, 10, 3, 4, 9, 2, 6, 5, 8, 7]",15788
7,"[2, 10, 4, 3, 7, 5, 9, 1, 6, 8]",16378
4,"[7, 6, 2, 1, 3, 5, 8, 10, 4, 9]",16425


## Partially Mapped Crossover

###### - implemented PMX conceptualized by goldberg

In [168]:
def crossOverFunction(parent1, parent2,crossover_factor_start_pos=2,
                             crossover_factor_end_pos=6):
    
    indexes_for_crossover = random.sample(set(range(len(parent1))), 2)
    
    crossover_factor_start_pos,crossover_factor_end_pos = min(indexes_for_crossover),max(indexes_for_crossover)
    print("Start"," End")
    print (crossover_factor_start_pos,"     ", crossover_factor_end_pos)
    
    ## generate child 1
    child1 = parent1[0:crossover_factor_start_pos]+\
    parent2[crossover_factor_start_pos:crossover_factor_end_pos] +\
    parent1[crossover_factor_end_pos:]
    # print(child1)
    # print("\n")
    
    ## generate child 2
    child2 = parent2[0:crossover_factor_start_pos] +\
    parent1[crossover_factor_start_pos:crossover_factor_end_pos] +\
    parent2[crossover_factor_end_pos:]
    
    # print(child2)
        
    ## Create mappings
    mapping = list(zip(parent1[crossover_factor_start_pos:crossover_factor_end_pos], # create tuple of mappings
                      parent2[crossover_factor_start_pos:crossover_factor_end_pos]))
    print("\n")
    print("Mapping")
    print(mapping)
    print("\n")
    
    # run until all the nodes in the route are unique
    while len(np.unique(child1)) != len(child1):
        child1_part = child1[:crossover_factor_start_pos]+child1[crossover_factor_end_pos:]
        
        # print(child1_part)
        # print("\n")
        for i in child1_part:
            for j in mapping:
                if i == j[1]: # j[1] - Parent 2 window elements                 
                    child1_part[child1_part.index(i)] = j[0] # j[0] - Parent 1 window elements
                    
        child1 = child1_part[:crossover_factor_start_pos]+ child1[crossover_factor_start_pos:crossover_factor_end_pos]+ child1_part[crossover_factor_start_pos:]
        
        print("Child1 Intermediate {}".format(child1))
    print("\n")
        
    # run until all the nodes in the route are unique
    while len(np.unique(child2)) != len(child2):
        child2_part = child2[:crossover_factor_start_pos]+child2[crossover_factor_end_pos:]
        # print(child2_part)
        for i in child2_part:
            for j in mapping:
                if i == j[0]:
                    child2_part[child2_part.index(i)] = j[1]
        child2 = child2_part[:crossover_factor_start_pos]+ child2[crossover_factor_start_pos:crossover_factor_end_pos]+ child2_part[crossover_factor_start_pos:]
        
        print("Child2 Intermediate {}".format(child2))
        
    return( child1,child2)


In [169]:
p1 = [5,7,2,4,8,9,3,6,1]

In [170]:
p2 = [8,6,9,3,2,5,1,7,4]

In [171]:
print(p1)
print(p2)

[5, 7, 2, 4, 8, 9, 3, 6, 1]
[8, 6, 9, 3, 2, 5, 1, 7, 4]


In [172]:
for i in zip([p1,p2],crossOverFunction(p1,p2)):
    print ("\nParent: {}  : Child: {}".format(i[0],i[1]))

Start  End
5       6


Mapping
[(9, 5)]


Child1 Intermediate [9, 7, 2, 4, 8, 5, 3, 6, 1]


Child2 Intermediate [8, 6, 5, 3, 2, 9, 1, 7, 4]

Parent: [5, 7, 2, 4, 8, 9, 3, 6, 1]  : Child: [9, 7, 2, 4, 8, 5, 3, 6, 1]

Parent: [8, 6, 9, 3, 2, 5, 1, 7, 4]  : Child: [8, 6, 5, 3, 2, 9, 1, 7, 4]


 ### Best Solution So Far


Total distance travelled to cover the final route of 
 Bangalore => Chennai => Hyderabad => Kolkata => Varanasi => Delhi => Agra => Jaipur => Udaipur => Mumbai => Mysore => Bangalore is 6815


In [173]:
def genetic_algorithm(sorted_current_generation, no_of_chromosomes = 10, mutation_factor=0.2, xover_factor = 0.3):

    
    nextgen_child = [] 
    
    xover_pool = int(round(no_of_chromosomes * xover_factor ))
    print("xover_pool=",xover_pool)
    
    no_of_loops = int((no_of_chromosomes // 2) )
    
    print('starting crossover')
    
    for i in range(0,no_of_loops):
        
        # print("sorted_current_generation")
        # print(sorted_current_generation)
        
        parent_index1 = random.choice(range(0,xover_pool))
        p1= sorted_current_generation.iloc[parent_index1,0]
        
        print("p1")
        print(p1)
        
        print("\n")
        
        parent_index2 = random.choice(range(0,xover_pool))
        p2= sorted_current_generation.iloc[parent_index2,0]
        
        print("p2")
        print(p2)
        
        c1, c2 = crossOverFunction(p1,p2)
        
        print("c1")
        print(c1)
        print("\n")
        print("c2")
        print(c2)
        print("\n")
        nextgen_child.extend([c1,c2])
    
    print("\n")
    print("nextgen_child")
    print(nextgen_child)
        
    nextgen_costs = [fitnessFunction(i) for i in nextgen_child]
    next_generation = pd.DataFrame([nextgen_child,nextgen_costs],).transpose() 
    next_generation.columns = ['Route','Cost']
    
    print("next_generation")
    print(next_generation)
    
    
    print('starting Mutation')
    
    for i in range(0, no_of_chromosomes) :
        mutation_prob = random.random() # using random() function to generate a random number between 0 and 1
        
        if mutation_prob < mutation_factor: 
            mutated_child = mutateRoute(next_generation.iloc[i,0] , 3)
            
            next_generation.iloc[i,0] = mutated_child
            
            # print("mutated_child")
            # print(next_generation.iloc[i,0])
            
            # print("\n")
            next_generation.iloc[i,1] = fitnessFunction(next_generation.iloc[i,0])
            
            # print("next_generation.iloc[i,1]")
            # print(next_generation.iloc[i,1])

    return (next_generation)

In [178]:
def perform_genetic_algorithm(no_of_iterations = 10, no_of_chromosomes = 300, mutation_factor=0.1, xover_factor = 0.5):
    best_solution = []
    best_cost = 99999
    current_generation = initialPopCost(no_of_chromosomes)
    sorted_current_generation = current_generation.sort_values('Cost')
    
    for i in range(0, no_of_iterations) :
        next_generation = genetic_algorithm(sorted_current_generation,no_of_chromosomes, mutation_factor, xover_factor)
        sorted_next_generation = next_generation.sort_values('Cost')
        
        print ("Completed {} runs of GA.".format(i + 1))
        print ("Below are the best chromosomes:")
        
        print(sorted_next_generation.head(2))

        print("\n")
        sorted_current_generation = sorted_next_generation
        if best_cost > sorted_next_generation.iloc[0,1] :
            best_cost = sorted_next_generation.iloc[0,1]
            best_solution = sorted_next_generation.iloc[0,0]
            print("best_solution")
            print(best_solution)
        
        final_route = []
        for i in best_solution:
            final_route.append(cities_mapping[i])
        final_route = [starting_point] + final_route + [starting_point]
        print("Total distance travelled to cover the final route of \n {} is {}".format(" => ".join(final_route),best_cost))

    return (best_solution, best_cost)


In [179]:
best_solution, best_cost = perform_genetic_algorithm()

routeCost+ 0 -> 8 1862


routeCost+ 8 -> 6 3874


routeCost+ 6 -> 9 4938


routeCost+ 9 -> 7 6261


routeCost+ 7 -> 1 6501


routeCost+ 1 -> 4 6788


routeCost+ 4 -> 10 8366


routeCost+ 10 -> 3 9715


routeCost+ 3 -> 5 11792


routeCost+ 5 -> 2 13654


routeCost+ 0 -> 3 1724


routeCost+ 3 -> 10 3073


routeCost+ 10 -> 4 4651


routeCost+ 4 -> 5 6854


routeCost+ 5 -> 6 7336


routeCost+ 6 -> 7 9408


routeCost+ 7 -> 1 9648


routeCost+ 1 -> 8 11168


routeCost+ 8 -> 9 13221


routeCost+ 9 -> 2 14745


routeCost+ 0 -> 5 346


routeCost+ 5 -> 4 2549


routeCost+ 4 -> 3 3210


routeCost+ 3 -> 6 5019


routeCost+ 6 -> 8 7031


routeCost+ 8 -> 10 8524


routeCost+ 10 -> 9 9235


routeCost+ 9 -> 7 10558


routeCost+ 7 -> 2 11172


routeCost+ 2 -> 1 12021


routeCost+ 0 -> 1 2026


routeCost+ 1 -> 10 3509


routeCost+ 10 -> 9 4220


routeCost+ 9 -> 2 5744


routeCost+ 2 -> 3 6898


routeCost+ 3 -> 8 8723


routeCost+ 8 -> 6 10735


routeCost+ 6 -> 4 13052


routeCost+ 4 -> 5 15255


routeCo

routeCost+ 1 -> 9 8487


routeCost+ 9 -> 5 9832


routeCost+ 5 -> 7 11789


routeCost+ 7 -> 6 13861


routeCost+ 6 -> 4 16178


routeCost+ 0 -> 10 569


routeCost+ 10 -> 2 1806


routeCost+ 2 -> 9 3330


routeCost+ 9 -> 8 5383


routeCost+ 8 -> 6 7395


routeCost+ 6 -> 4 9712


routeCost+ 4 -> 3 10373


routeCost+ 3 -> 5 12450


routeCost+ 5 -> 1 14558


routeCost+ 1 -> 7 14798


routeCost+ 0 -> 6 149


routeCost+ 6 -> 10 867


routeCost+ 10 -> 2 2104


routeCost+ 2 -> 7 2718


routeCost+ 7 -> 1 2958


routeCost+ 1 -> 4 3245


routeCost+ 4 -> 5 5448


routeCost+ 5 -> 8 7117


routeCost+ 8 -> 3 8942


routeCost+ 3 -> 9 9695


routeCost+ 0 -> 5 346


routeCost+ 5 -> 9 1691


routeCost+ 9 -> 7 3014


routeCost+ 7 -> 10 4348


routeCost+ 10 -> 4 5926


routeCost+ 4 -> 2 6760


routeCost+ 2 -> 3 7914


routeCost+ 3 -> 8 9739


routeCost+ 8 -> 6 11751


routeCost+ 6 -> 1 13872


routeCost+ 0 -> 9 980


routeCost+ 9 -> 8 3033


routeCost+ 8 -> 10 4526


routeCost+ 10 -> 6 5244


routeCost+ 6 

routeCost+ 0 -> 8 1862


routeCost+ 8 -> 4 3360


routeCost+ 4 -> 1 3647


routeCost+ 1 -> 6 5768


routeCost+ 6 -> 9 6832


routeCost+ 9 -> 10 7543


routeCost+ 10 -> 3 8892


routeCost+ 3 -> 5 10969


routeCost+ 5 -> 2 12831


routeCost+ 2 -> 7 13445


routeCost+ 0 -> 9 980


routeCost+ 9 -> 3 1733


routeCost+ 3 -> 7 2367


routeCost+ 7 -> 4 2600


routeCost+ 4 -> 8 4098


routeCost+ 8 -> 10 5591


routeCost+ 10 -> 1 7074


routeCost+ 1 -> 5 9182


routeCost+ 5 -> 2 11044


routeCost+ 2 -> 6 13020


routeCost+ 0 -> 2 1825


routeCost+ 2 -> 4 2659


routeCost+ 4 -> 8 4157


routeCost+ 8 -> 3 5982


routeCost+ 3 -> 7 6616


routeCost+ 7 -> 1 6856


routeCost+ 1 -> 9 8002


routeCost+ 9 -> 5 9347


routeCost+ 5 -> 6 9829


routeCost+ 6 -> 10 10547


routeCost+ 0 -> 2 1825


routeCost+ 2 -> 7 2439


routeCost+ 7 -> 9 3762


routeCost+ 9 -> 3 4515


routeCost+ 3 -> 8 6340


routeCost+ 8 -> 6 8352


routeCost+ 6 -> 1 10473


routeCost+ 1 -> 10 11956


routeCost+ 10 -> 5 12583


routeCost+

routeCost+ 9 -> 5 2325


routeCost+ 5 -> 6 2807


routeCost+ 6 -> 10 3525


routeCost+ 10 -> 4 5103


routeCost+ 4 -> 2 5937


routeCost+ 2 -> 1 6786


routeCost+ 1 -> 8 8306


routeCost+ 8 -> 7 9586


routeCost+ 7 -> 3 10220


routeCost+ 0 -> 9 980


routeCost+ 9 -> 3 1733


routeCost+ 3 -> 8 3558


routeCost+ 8 -> 10 5051


routeCost+ 10 -> 6 5769


routeCost+ 6 -> 4 8086


routeCost+ 4 -> 7 8319


routeCost+ 7 -> 5 10276


routeCost+ 5 -> 1 12384


routeCost+ 1 -> 2 13233


routeCost+ 0 -> 4 2166


routeCost+ 4 -> 6 4483


routeCost+ 6 -> 1 6604


routeCost+ 1 -> 8 8124


routeCost+ 8 -> 5 9793


routeCost+ 5 -> 2 11655


routeCost+ 2 -> 10 12892


routeCost+ 10 -> 3 14241


routeCost+ 3 -> 7 14875


routeCost+ 7 -> 9 16198


routeCost+ 0 -> 3 1724


routeCost+ 3 -> 2 2878


routeCost+ 2 -> 10 4115


routeCost+ 10 -> 5 4742


routeCost+ 5 -> 1 6850


routeCost+ 1 -> 6 8971


routeCost+ 6 -> 7 11043


routeCost+ 7 -> 9 12366


routeCost+ 9 -> 4 13781


routeCost+ 4 -> 8 15279


route

routeCost+ 0 -> 7 1923


routeCost+ 7 -> 5 3880


routeCost+ 5 -> 3 5957


routeCost+ 3 -> 2 7111


routeCost+ 2 -> 6 9087


routeCost+ 6 -> 8 11099


routeCost+ 8 -> 10 12592


routeCost+ 10 -> 9 13303


routeCost+ 9 -> 4 14718


routeCost+ 4 -> 1 15005


routeCost+ 0 -> 10 569


routeCost+ 10 -> 6 1287


routeCost+ 6 -> 9 2351


routeCost+ 9 -> 1 3497


routeCost+ 1 -> 3 3890


routeCost+ 3 -> 7 4524


routeCost+ 7 -> 8 5804


routeCost+ 8 -> 5 7473


routeCost+ 5 -> 4 9676


routeCost+ 4 -> 2 10510


routeCost+ 0 -> 7 1923


routeCost+ 7 -> 8 3203


routeCost+ 8 -> 6 5215


routeCost+ 6 -> 5 5697


routeCost+ 5 -> 10 6324


routeCost+ 10 -> 9 7035


routeCost+ 9 -> 3 7788


routeCost+ 3 -> 4 8449


routeCost+ 4 -> 1 8736


routeCost+ 1 -> 2 9585


routeCost+ 0 -> 5 346


routeCost+ 5 -> 9 1691


routeCost+ 9 -> 7 3014


routeCost+ 7 -> 6 5086


routeCost+ 6 -> 4 7403


routeCost+ 4 -> 1 7690


routeCost+ 1 -> 2 8539


routeCost+ 2 -> 3 9693


routeCost+ 3 -> 8 11518


routeCost+ 8 -

...

routeCost+ 6 -> 5 631


routeCost+ 5 -> 8 2300


routeCost+ 8 -> 1 3820


routeCost+ 1 -> 4 4107


routeCost+ 4 -> 2 4941


routeCost+ 2 -> 7 5555


routeCost+ 7 -> 3 6189


routeCost+ 3 -> 9 6942


routeCost+ 9 -> 10 7653


routeCost+ 0 -> 6 149


routeCost+ 6 -> 5 631


routeCost+ 5 -> 4 2834


routeCost+ 4 -> 2 3668


routeCost+ 2 -> 8 4351


routeCost+ 8 -> 7 5631


routeCost+ 7 -> 1 5871


routeCost+ 1 -> 3 6264


routeCost+ 3 -> 10 7613


routeCost+ 10 -> 9 8324


routeCost+ 0 -> 6 149


routeCost+ 6 -> 10 867


routeCost+ 10 -> 8 2360


routeCost+ 8 -> 2 3043


routeCost+ 2 -> 9 4567


routeCost+ 9 -> 4 5982


routeCost+ 4 -> 3 6643


routeCost+ 3 -> 1 7036


routeCost+ 1 -> 7 7276


routeCost+ 7 -> 5 9233


routeCost+ 0 -> 5 346


routeCost+ 5 -> 10 973


routeCost+ 10 -> 9 1684


routeCost+ 9 -> 7 3007


routeCost+ 7 -> 4 3240


routeCost+ 4 -> 1 3527


routeCost+ 1 -> 2 4376


routeCost+ 2 -> 8 5059


routeCost+ 8 -> 3 6884


routeCost+ 3 -> 6 8693


routeCost+ 0 -> 5 346


r

routeCost+ 4 -> 7 3494


routeCost+ 7 -> 3 4128


routeCost+ 3 -> 8 5953


routeCost+ 8 -> 9 8006


routeCost+ 9 -> 5 9351


routeCost+ 5 -> 10 9978


routeCost+ 0 -> 5 346


routeCost+ 5 -> 9 1691


routeCost+ 9 -> 3 2444


routeCost+ 3 -> 1 2837


routeCost+ 1 -> 4 3124


routeCost+ 4 -> 7 3357


routeCost+ 7 -> 8 4637


routeCost+ 8 -> 2 5320


routeCost+ 2 -> 10 6557


routeCost+ 10 -> 6 7275


routeCost+ 0 -> 6 149


routeCost+ 6 -> 5 631


routeCost+ 5 -> 8 2300


routeCost+ 8 -> 3 4125


routeCost+ 3 -> 1 4518


routeCost+ 1 -> 2 5367


routeCost+ 2 -> 7 5981


routeCost+ 7 -> 4 6214


routeCost+ 4 -> 9 7629


routeCost+ 9 -> 10 8340


routeCost+ 0 -> 5 346


routeCost+ 5 -> 10 973


routeCost+ 10 -> 4 2551


routeCost+ 4 -> 2 3385


routeCost+ 2 -> 8 4068


routeCost+ 8 -> 7 5348


routeCost+ 7 -> 1 5588


routeCost+ 1 -> 3 5981


routeCost+ 3 -> 6 7790


routeCost+ 6 -> 9 8854


routeCost+ 0 -> 6 149


routeCost+ 6 -> 10 867


routeCost+ 10 -> 7 2201


routeCost+ 7 -> 1 2441





routeCost+ 5 -> 10 973


routeCost+ 10 -> 6 1691


routeCost+ 6 -> 3 3500


routeCost+ 3 -> 4 4161


routeCost+ 4 -> 8 5659


routeCost+ 8 -> 2 6342


routeCost+ 2 -> 1 7191


routeCost+ 1 -> 7 7431


routeCost+ 7 -> 9 8754


routeCost+ 0 -> 6 149


routeCost+ 6 -> 5 631


routeCost+ 5 -> 7 2588


routeCost+ 7 -> 3 3222


routeCost+ 3 -> 4 3883


routeCost+ 4 -> 1 4170


routeCost+ 1 -> 8 5690


routeCost+ 8 -> 2 6373


routeCost+ 2 -> 9 7897


routeCost+ 9 -> 10 8608


routeCost+ 0 -> 5 346


routeCost+ 5 -> 10 973


routeCost+ 10 -> 7 2307


routeCost+ 7 -> 3 2941


routeCost+ 3 -> 4 3602


routeCost+ 4 -> 1 3889


routeCost+ 1 -> 8 5409


routeCost+ 8 -> 2 6092


routeCost+ 2 -> 9 7616


routeCost+ 9 -> 6 8680


routeCost+ 0 -> 9 980


routeCost+ 9 -> 10 1691


routeCost+ 10 -> 4 3269


routeCost+ 4 -> 7 3502


routeCost+ 7 -> 2 4116


routeCost+ 2 -> 3 5270


routeCost+ 3 -> 1 5663


routeCost+ 1 -> 8 7183


routeCost+ 8 -> 5 8852


routeCost+ 5 -> 6 9334


routeCost+ 0 -> 10 569

routeCost+ 3 -> 8 4156


routeCost+ 8 -> 2 4839


routeCost+ 2 -> 7 5453


routeCost+ 7 -> 1 5693


routeCost+ 1 -> 4 5980


routeCost+ 4 -> 5 8183


routeCost+ 0 -> 6 149


routeCost+ 6 -> 3 1958


routeCost+ 3 -> 4 2619


routeCost+ 4 -> 7 2852


routeCost+ 7 -> 1 3092


routeCost+ 1 -> 2 3941


routeCost+ 2 -> 9 5465


routeCost+ 9 -> 8 7518


routeCost+ 8 -> 10 9011


routeCost+ 10 -> 5 9638


routeCost+ 0 -> 6 149


routeCost+ 6 -> 2 2125


routeCost+ 2 -> 7 2739


routeCost+ 7 -> 3 3373


routeCost+ 3 -> 4 4034


routeCost+ 4 -> 1 4321


routeCost+ 1 -> 10 5804


routeCost+ 10 -> 8 7297


routeCost+ 8 -> 9 9350


routeCost+ 9 -> 5 10695


routeCost+ 0 -> 6 149


routeCost+ 6 -> 3 1958


routeCost+ 3 -> 8 3783


routeCost+ 8 -> 7 5063


routeCost+ 7 -> 4 5296


routeCost+ 4 -> 1 5583


routeCost+ 1 -> 9 6729


routeCost+ 9 -> 2 8253


routeCost+ 2 -> 10 9490


routeCost+ 10 -> 5 10117


routeCost+ 0 -> 6 149


routeCost+ 6 -> 10 867


routeCost+ 10 -> 2 2104


routeCost+ 2 -> 4 29

routeCost+ 6 -> 10 8886


routeCost+ 10 -> 9 9597


[6, 10, 8]
[7, 2, 1, 3, 9, 4, 5]
[7, 2, 1, 3, 9, 4, 5, 6, 10, 8]
routeCost+ 0 -> 7 1923


routeCost+ 7 -> 2 2537


routeCost+ 2 -> 1 3386


routeCost+ 1 -> 3 3779


routeCost+ 3 -> 9 4532


routeCost+ 9 -> 4 5947


routeCost+ 4 -> 5 8150


routeCost+ 5 -> 6 8632


routeCost+ 6 -> 10 9350


routeCost+ 10 -> 8 10843


[6, 8, 4]
[2, 7, 1, 3, 9, 10, 5]
[2, 7, 1, 3, 9, 10, 5, 6, 8, 4]
routeCost+ 0 -> 2 1825


routeCost+ 2 -> 7 2439


routeCost+ 7 -> 1 2679


routeCost+ 1 -> 3 3072


routeCost+ 3 -> 9 3825


routeCost+ 9 -> 10 4536


routeCost+ 10 -> 5 5163


routeCost+ 5 -> 6 5645


routeCost+ 6 -> 8 7657


routeCost+ 8 -> 4 9155


[5, 10, 9]
[4, 8, 2, 3, 1, 7, 6]
[4, 8, 2, 3, 1, 7, 6, 5, 10, 9]
routeCost+ 0 -> 4 2166


routeCost+ 4 -> 8 3664


routeCost+ 8 -> 2 4347


routeCost+ 2 -> 3 5501


routeCost+ 3 -> 1 5894


routeCost+ 1 -> 7 6134


routeCost+ 7 -> 6 8206


routeCost+ 6 -> 5 8688


routeCost+ 5 -> 10 9315


routeCost+ 10 -> 9 100

In [180]:
print(best_solution)
print(best_cost)

[5, 8, 2, 4, 7, 1, 3, 9, 6, 10]
7502


In [181]:
final_route = []
for i in best_solution:
    final_route.append(cities_mapping[i])
    
final_route = [starting_point] + final_route + [starting_point]

print("Total distance travelled to cover the final route of \n {} is {}".format(" => ".join(final_route),best_cost))


Total distance travelled to cover the final route of 
 Bangalore => Chennai => Kolkata => Varanasi => Delhi => Agra => Jaipur => Udaipur => Mumbai => Mysore => Hyderabad => Bangalore is 7502
